# 1. Preliminaries
for each subject, we have a directory with multiple `.mat` files stored. All subject folders are stored in folder named `'logs'`. Output of the code in this notebook will be stored to the directory specified in the variabel `outputdir` which is `'outputs'` by default.

In [1]:
clear
logdir = 'logs';
outputdir = 'outputs';
p_w_d  = pwd;

Besides the code in these jupyter notebooks, there are some custom helper functions provided in this repository. These functions are stored in a folder named 'helperFunctions', which we need to add to the path:

In [2]:
addpath('helperFunctions')

We use the Palamedes Toolbox not only to for adaptive algorithms during data collection, but also to fit psychometric curves to the data as e.g. in Figure 3.

Prins, N & Kingdom, F. A. A. (2009) Palamedes:  Matlab routines for analyzing psychophysical data. http://www.palamedestoolbox.org 

We used version 1.5.0 but more recent version should work as well. Once downloaded to your machine trom, add the function to the matlab path as well.

In [3]:
% palamedespath = ['Path' filesep 'to' filesep 'Palamedes'];
% e.g.: 
palamedespath = '/Users/treber/projects/matlab_common/palamedes1_5_0/Palamedes'; 
addpath(palamedespath)

A Reviewer asked to compute d' with corrected Hits and False ass suggested in 

Snodgrass, Joan G., and June Corwin. “Pragmatics of Measuring Recognition Memory: Applications to Dementia and Amnesia.” Journal of Experimental Psychology: General 117, no. 1 (1988): 34.

This can be done by setting `hfa_correction = 'snodgrass&corwin1988'`.  
By default `'hfa_correction'` is set to `'regular'`, which results in replacing hits and false alarm rates of exactly zero or one to 0.01 and 0.99 respectively.


In [4]:
hfacorrection = 'regular';
%hfacorrection = 'snodgrass&corwin1988';

# 2. Load Data in Workspace
In this part, we use some of these helper functions to aggregate the data from different parts of the experiment and from different subject.
### 2.1 Proportion of GO responses in the conditioning part
Calling `sc_average_p_go.m` returns `p_go`, a matrix of $N_{S} \times N_{V} \times N_{R}$, where $N_{s}$ is the number of participants, $N_{V}$ the number of visibility levels, i.e., runs in the conditioning experiment, and $N_{R}$ the reward levels, i.e., 'rewarding cue'(1) or 'punishing cue'(2). Thus, by `p_go(3,2,1)` we access the proportion of go responses in the third participant in the second run (@th) following the rewarding cue.

It also returns the mean and it standard error of the proportion of go responses for each of the four runs (`p_go_avg` and `p_go_sem`).

In [5]:
[p_go_avg p_go_sem p_go]=sc_average_p_go(logdir, 0,hfacorrection);

### 2.2. Data from individual trials of conditioning part
Calling `sc_aggregate_data.m` also gets data from all four runs of the conditionign experiment, but we get data from all trials of all subjects. Here, the data is stored in a cell array for each run.

In [6]:
conditioning = sc_aggregate_data(logdir);

In each entry of this cell array, there is a struct with different variables recorded by psychtoolbox. Lets have a look at variables stored during the first run (which are identical to runs 2 to 4).

In [7]:
run = 1;
fieldnames(conditioning{run})


ans =

  24×1 cell array

    'condition'
    'threshold'
    'subject'
    'run'
    'th'
    'reward'
    'money'
    'gain'
    'initial_reward'
    'stim'
    'stim_event'
    'trial_onset'
    'forward_mask_onset'
    'stimulus_onset'
    'backward_mask_onset'
    'blank_onset'
    'go_nogo_screen_onset'
    'go_rt'
    'go_response'
    'go_response_time'
    'go_choice_feedback_screen_onset'
    'rewarding_feedback_screen_onset'
    'nogo_choice_feedback_screen_onset'
    'neutral_feedback_screen_onset'



each of these variables is of size $N_{S} \times N_{T}$, where $N_{s}$ is the number of participants, $N_{T}$ is the number of trials.

In [8]:
[nsubs, ntrials] = size(conditioning{run}.condition)


nsubs =

    29


ntrials =

    16



### 2.3. Data from the subjective visibility ratings performed after each run of the conditioning experiment.
Lets aggreagate subjective visibility ratings by using `sc_aggergate_inverview_data.m`:

In [9]:
interviews = sc_aggregate_interview_data(logdir);

> In sc_aggregate_interview_data (line 29)
> In sc_aggregate_interview_data (line 29)


The warning messages reminds us that, interview data was lost for two subjects. 

The follwoing demonstrates how subjective awareness ratings can be accessed:

In [10]:
subject = 12;
run = 2;

disp(sprintf('Partcipant #%d reported having seen the cue in %.0f%% of the trials during run %d', ...
subject, interviews{subject}.awareness_rating(run)*10, run))

Partcipant #12 reported having seen the cue in 70% of the trials during run 2


### 2.4. Data from the threshold estimation procedures preceding and following the conditioning part.
Lets aggreagate these data by using `sc_aggregate_th_data.m`:

In [11]:
[th1 th2] = sc_aggregate_th_data(logdir);

Similar to single-trial data in the conditiong part above, data is stored in a struct, where ech field in the struct is matrix of size $N_{S} \times N_{T}$, where $N_{s}$ is the number of participants, $N_{T}$ is the number of trials.

In [12]:
fieldnames(th1)
[nsubs, ntrials] = size(th1.correct)


ans =

  25×1 cell array

    'trial_onset'
    'forward_mask_onset'
    'stimulus_onset'
    'backward_mask_onset'
    'blank_onset'
    'stimulus_choice_onset'
    'stim_choice_rt'
    'stim_choice_button'
    'chosen_stimulus'
    'chosen_stim_str'
    'correct'
    'seen_decision_onset'
    'seen_decision_response'
    'seen_decision_rt'
    'seen'
    'stim_nr'
    'stim_event'
    'noise_variance'
    'th'
    'subject'
    'run'
    'seen_button'
    'unseen_button'
    'presented_stim_str'
    'elapsed_time'


nsubs =

    29


ntrials =

   192



Visibility of masked images in the experiment was varied by adding noise. The amount of noise is given by the variance of the noise distirbution we can vary from 0, i.e., no nose to 2.5 (maximum noise level). Thus more noise means less visible and vice versa. But we prefer to think in terms of stimulus intensity. Therefore, we compute 2.5 - noise variance, to quantifiy the visibility of a stimulus.

In [13]:
th1 = sc_reverse_noise_variance(th1);
th2 = sc_reverse_noise_variance(th2);

### 2.5 Get d primes during pre and post threshold tests
We use the helper funtion `sc_get_d_primes.m`, which loads data, and calls `dprime.m`. Data is returned in array of $N$ subjects (1 to 29) $\times$ $N$ threshold tests (pre, post) $\times$ $N$ visibility levels (1 to 4) $\times$ $N$ stimuli (1 to 6).

In [14]:
[th_test_dprimes stimlookup]= sc_get_d_primes(logdir, hfacorrection);
%th_test_dprimes(subject,th-test 1vs2, run, stimulus)
[nsubs, nthtests, nlevels, nstimuli] = size(th_test_dprimes)


nsubs =

    29


nthtests =

     2


nlevels =

     4


nstimuli =

     6



### 2.6 Aggregate Data on the level of individual stimuli
This form of the data will be used for plots in Figure 3.

In [15]:
[perstim perstim_mean] = sc_aggregate_per_stimulus(conditioning, th1, th2);
perstim


perstim = 

  8×4 struct array with fields:

    name
    correct_pre
    correct_r
    correct_p
    intensity_pre
    intensity_r
    intensity_p
    subj_r
    subj_p
    subj_pre



# 3. Rearrange Data in "long-format"
Most of the statistics reported in the manuscript have been calcualted using R. These analyses are documented in R jupyter notebook (`R_Analyses.ipynb`). Lets rearrange the data in a format R can access:

In [16]:
th_levels =         [2 1 0.5 0]; % >th, @th, <th, <<th
visibility_levels = [4 3 2   1];
runs =              [1 2 3   4];
counter = 1; % line counter in output


for s=1:nsubs
    for run_ = runs
        
        visibility_ = visibility_levels(run_);
        
        is_ =find(th2.th(s,:) == th_levels(run_));
        seen_unseen(counter) = sum(th2.seen(s,is_))/length(is_);
        
        % pgo
        p_go_r(counter) = p_go(s,run_,1);
        p_go_p(counter) = p_go(s,run_,2); 
        
        % pgo d-prime
        dprime_ = dprime(p_go_r(counter), p_go_p(counter));
       
        dprime_conditioning(counter) = dprime_;
        subjective_awareness(counter) = interviews{s}.awareness_rating(run_);
        
        % get average dprime at th-level across stimuli
        dprime_post_av(counter) = mean(th_test_dprimes(s,2,visibility_, :));
        
        % get rewarding/punishing stim 
        is = find(conditioning{run_}.condition(s,:) == 1);
        stim_r{counter} = conditioning{run_}.stim{s,is(1)};
        is = find(conditioning{run_}.condition(s,:) == 2);
        stim_p{counter} = conditioning{run_}.stim{s,is(1)};
        
        if run_ > 1
            % get th-test 2 dprimes for stimuli used in conditioning
            is = find(strcmp(stim_r{counter}, stimlookup{s,2,:}));
            dprime_post_r(counter) = th_test_dprimes(s,2,visibility_,is);
            
            is = find(strcmp(stim_p{counter}, stimlookup{s,2,:}));
            dprime_post_p(counter) = th_test_dprimes(s,2,visibility_,is);
            
            % get alphas from th-test1, th-test2, separate for p and r
            is = strcmp(stim_r{counter}, th1.presented_stim_str(s,:));            
            alpha_pre_r(counter) = fit_curve(th1.intensity(s,is), ...
                                             th1.correct(s,is));
            
            is = strcmp(stim_p{counter}, th1.presented_stim_str(s,:));            
            alpha_pre_p(counter) = fit_curve(th1.intensity(s,is), ...
                                             th1.correct(s,is));

            is = strcmp(stim_r{counter}, th2.presented_stim_str(s,:));            
            alpha_post_r(counter) = fit_curve(th2.intensity(s,is), ...
                                             th2.correct(s,is));
            
            is = strcmp(stim_p{counter}, th1.presented_stim_str(s,:));            
            alpha_post_p(counter) = fit_curve(th2.intensity(s,is), ...
                                              th2.correct(s,is));

        else % th-test was not performed for stimuli used in run 1 (>th)
            dprime_post_p(counter) = NaN;
            dprime_post_r(counter) = NaN;
            alpha_pre_r(counter) = NaN;
            alpha_pre_p(counter) = NaN;
            alpha_post_r(counter) = NaN;
            alpha_post_p(counter) = NaN;            
        end
                
        % design vars, subject counter
        S(counter) = s;
        visibility(counter) = visibility_;
        run(counter) = run_;
        
        switch(run_)
          case 4 % > th
            dummy1(counter) = 0;
            dummy2(counter) = 0;
            dummy3(counter) = 1;
          case 3 % @ th
            dummy1(counter) = 0;
            dummy2(counter) = 1;
            dummy3(counter) = 0;
          case 2 % < th
            dummy1(counter) = 1;
            dummy2(counter) = 0;
            dummy3(counter) = 0;
          otherwise % <<th
            dummy1(counter) = 0;
            dummy2(counter) = 0;
            dummy3(counter) = 0;
        end
                
        counter = counter + 1;
        
    end
end

### Save Data for Further Processing

In [17]:
% change into output directory
cd(outputdir);

% save for later use in matlab
visibility_string = {'well_below_th', 'below_th', 'at_th', 'above_th'};
save (['sc_' hfacorrection '.mat'],'S', 'visibility', 'visibility_string', 'run', 'dummy1', 'dummy2', ...
    'dummy3', 'p_go_r', 'p_go_p', 'dprime_conditioning', 'stim_r', ...
    'stim_p', 'dprime_post_r', 'dprime_post_p', 'alpha_pre_r', 'alpha_pre_p',  ...
    'alpha_post_r', 'alpha_post_p', 'dprime_post_av', 'subjective_awareness', ... 
    'th1', 'th2', 'perstim', 'perstim_mean', 'hfacorrection')

% save for use with R

d = ','; % delimiter
filename =  ['sc_' hfacorrection, '.txt'];
header = {'subject', ...
          'visibility', ... % 1: low  4: high
          'Visibility', .... % a string representation of the above 
          'run', ...
          'belowth_vs_invisible', ...
          'threshold_vs_invisible', ...
          'visible_vs_invisble',...
          'pGOr', ...
          'pGOp', ...
          'dprime_conditioning', ...
          'stim_r', ...
          'stim_p', ...
          'd_post_r', ...
          'd_post_p', ...
          'alpha_pre_r', ...
          'alpha_pre_p', ...
          'alpha_post_r', ...
          'alpha_post_p', ...
          'dprime_post_av', ...
          'subjective_awareness', ...
          'seen_unseen';};

format_string = ['%d' d ... % subject
                 '%d' d ... % visibility numeric
                 '%s' d ... % visibility string
                 '%d' d ... % run
                 '%d' d ... % dummy1 visiblity
                 '%d' d ... % dummy2
                 '%d' d ... % dummy3
                 '%.3f' d ... % pGO r
                 '%.3f' d ... % pGO p
                 '%.3f' d ... % dprime conditionign
                 '%s' d ... % stim r
                 '%s' d ... % stim p
                 '%.3f' d ... % dprime post-r
                 '%.3f' d ... % dprime post-p
                 '%.3f' d ... % alpha pre-r
                 '%.3f' d ... % alpha pre-p
                 '%.3f' d ... % alpha post-r
                 '%.3f' d ... % alpha post-p
                 '%.3f' d ... % dprime averaged over all stimuli
                 '%d' d ... subjective awareness
                 '%d' d]; % seen_unseen 

try
    
    fid = fopen(filename, 'w');
    h_string = sprintf(['%s' d],header{:}); 
    fprintf(fid,'%s',h_string(1:end-1)); 
    fprintf(fid, '\n');

    for i = 1:length(S)
        
        fprintf(fid, format_string, ...
                S(i), ...
                visibility(i), ...
                visibility_string{visibility(i)}, ...
                run(i), ...
                dummy1(i), ...
                dummy2(i), ...
                dummy3(i), ...
                p_go_r(i), ...
                p_go_p(i), ...
                dprime_conditioning(i), ...
                stim_r{i}, ...
                stim_p{i}, ...
                dprime_post_r(i), ...
                dprime_post_p(i), ...
                alpha_pre_r(i), ...
                alpha_pre_p(i), ...
                alpha_post_r(i), ...
                alpha_post_p(i), ...
                dprime_post_av(i),...
                subjective_awareness(i));
        fprintf(fid, '\n');
    end
catch err
    fclose(fid);
    err
end

fclose(fid);

% change directory back to where it was in the beginning of the script
cd(p_w_d)


# 4. Aggregate and save data for 2 way repeated measures ANOVA 

In [18]:
clear conditioning p_go
dirs = dir([p_w_d, filesep, logdir, filesep, 'sc*']);
nsubs = numel(dirs);
disp(sprintf('Found %d data-directories', nsubs))

% intiailize a subject counter
ss_counter = 1;   

% initialize data structure for aggregated data of conditioning experiment
conditioning.run = [];
conditioning.subject = [];
conditioning.condition = [];
coniditoning.p_go = [];
p_go = [];
ii = 1; % a counter for the output struct

for d = 1:length(dirs)
    
    % load mat files for each of the four conditioning runs
    for r = 1:4
        clear data;
        fname = dir(sprintf('%s%s%s%s%s%d%s',dirs(d).folder, filesep, dirs(d).name, filesep, '*run_',r,'*'));
        load([fname.folder filesep fname.name]);
        
        % loop over rewarding (1) and punishing (2) cues
        for cc = 1:2
            % get proportion of GO responses 
            % for later plotting:
            
            nGo = sum(data.condition == cc & data.go_response == 1);
            nTrials = sum(data.condition == cc);

            if strcmp(hfacorrection, 'snodgrass&corwin1988')
                pg = (nGo+0.5)/(nTrials+1);
            else
                pg = nGo/nTrials;
                if pg == 1
                    pg = 0.99;
                end
                if pg == 0
                    pg = 0.01;
                end    
            end
            p_go(ss_counter, r, cc) = pg;
            
            % store also in long-format for stats
            conditioning.p_go(ii) = p_go(ss_counter, r, cc);
            conditioning.run(ii) = r;
            conditioning.subject(ii) = ss_counter;
            conditioning.condition(ii) = cc;
            
            ii = ii + 1;
        end 
        clear data;
    end
    ss_counter = ss_counter + 1;
end

Found 29 data-directories


### save data for later use R and Matlab

In [19]:
cd(outputdir)
delimiter = ',';
filename =  ['visibilityXreward' hfacorrection, '.txt'];
header = ['Pgo', delimiter , 'subject', delimiter, 'visibility', delimiter 'reward'];
fid = fopen(filename, 'w');
fprintf(fid,'%s',header); 
fprintf(fid, '\n');
fclose(fid);
dlmwrite(filename, [conditioning.p_go', conditioning.subject', conditioning.run', conditioning.condition'], 'delimiter', ...
         delimiter, 'precision', 5, '-append');
save (['visibilityXreward' hfacorrection '.mat'], 'conditioning',  'p_go',  'hfacorrection');
cd(p_w_d)